In [ ]:

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import sqlite3  # Using sqlite3 for database operations

# Function to log progress
def log_progress(message):
    """This function logs the mentioned message of a given stage of code execution to a log file. Function returns nothing."""
    timestamp_format = '%Y-%m-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("code_log.txt", 'a') as f:
        f.write(f"{timestamp}: {message}\n")

# Function to extract data
def extract(url, table_attribs):
    """Extracts data from the given URL and returns a DataFrame."""
    log_progress("Starting data extraction.")
    try:
        data = requests.get(url).text
        html = BeautifulSoup(data, 'html.parser')
        df = pd.DataFrame(columns=table_attribs)
        tables = html.find_all('tbody')
        rows = tables[0].find_all('tr')
        for row in rows:
            col = row.find_all('td')
            if len(col) != 0:
                if col[1].find('a') is not None:
                    links = col[1].find_all('a')
                    data_dict = {
                        'Name': links[1].contents[0],
                        'MC_USD_Billion': col[2].contents[0].strip()
                    }
                    df1 = pd.DataFrame(data_dict, index=[0])
                    df = pd.concat([df, df1], ignore_index=True)
        log_progress("Data extraction completed.")
        return df
    except Exception as e:
        log_progress(f"Error during extraction: {e}")
        raise

# Function to transform data
def transform(df, csv_path):
    """Transforms the extracted data into the required format."""
    log_progress("Starting data transformation.")
    try:
        dataframe = pd.read_csv(csv_path)
        currency_dict = dataframe.set_index('Currency').to_dict()['Rate']
        USD_list = df["MC_USD_Billion"].tolist()
        USD_list = [float("".join(x.split(','))) for x in USD_list]
        df['MC_GBP_Billion'] = [np.round(x * currency_dict['GBP'], 2) for x in USD_list]
        df['MC_EUR_Billion'] = [np.round(x * currency_dict['EUR'], 2) for x in USD_list]
        df['MC_INR_Billion'] = [np.round(x * currency_dict['INR'], 2) for x in USD_list]
        log_progress("Data transformation completed.")
        return df
    except Exception as e:
        log_progress(f"Error during transformation: {e}")
        raise

# Function to save the data to a CSV file
def load_to_csv(df, output_path):
    """Saves the final data frame as a CSV file."""
    log_progress("Saving data to CSV.")
    try:
        df.to_csv(output_path, index=False)
        log_progress("Data successfully saved to CSV.")
    except Exception as e:
        log_progress(f"Error saving to CSV: {e}")
        raise

# Function to load the data into a SQLite database table
def load_to_db(df, db_path, table_name):
    """Saves the final data frame to a database table."""
    log_progress("Saving data to database.")
    try:
        with sqlite3.connect(db_path) as conn:
            df.to_sql(table_name, conn, if_exists='replace', index=False)
        log_progress("Data successfully saved to database.")
    except Exception as e:
        log_progress(f"Error saving to database: {e}")
        raise

# Function to run SQL queries
def run_queries(db_path):
    """Runs predefined SQL queries on the database."""
    log_progress("Running SQL queries.")
    try:
        queries = [
            "SELECT * FROM Largest_banks",
            "SELECT AVG(MC_GBP_Billion) FROM Largest_banks",
            "SELECT Name FROM Largest_banks LIMIT 5"
        ]
        with sqlite3.connect(db_path) as conn:
            for query in queries:
                result = pd.read_sql_query(query, conn)
                print(f"Query: {query}\nResult:\n{result}\n")
        log_progress("SQL queries executed successfully.")
    except Exception as e:
        log_progress(f"Error executing queries: {e}")
        raise

# Main execution
if __name__ == "__main__":
    # Define variables
    url = "https://example.com/banks"  # Replace with actual URL
    table_attribs = ["Name", "MC_USD_Billion"]
    csv_path = "currency_rates.csv"  # Path to currency rate CSV
    output_csv_path = "largest_banks.csv"
    db_path = "banks.db"  # SQLite database file path
    table_name = "Largest_banks"

    try:
        # ETL pipeline
        extracted_data = extract(url, table_attribs)
        transformed_data = transform(extracted_data, csv_path)
        load_to_csv(transformed_data, output_csv_path)
        load_to_db(transformed_data, db_path, table_name)
        
        # Running queries
        run_queries(db_path)
    except Exception as e:
        log_progress(f"Error in ETL pipeline: {e}")

